<a href="https://colab.research.google.com/github/NUG30/homework-2-mell-crane/blob/main/Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework 2: Naive Bayes Email Spam Filter

In this homework, we will learn how to implement the Naive Bayes classifier in order to create a simple email spam filter. This spam filter will be trained by test_emails, which will be given by a vector of tuples (emails, spam/nospam). For each tuple the first entry is a string ("email"), and the second entry is 0 or 1, depending whether the email contains spam words.

In [182]:
import numpy as np
import pandas as pd

In [183]:
dictionary = ['hello', 'students', 'please', 'learn', 'for', 'the', 'exam', 'buy', 'drugs', 
              'today', 'sun', 'is', 'shining', 'in', 'nagoya', 'lets', 'sell', 'how', 'are', 'you', 
              'today?', 'do', 'your', 'homework', 'want', 'free', 'solutions?', 'hey', 'always', 
              'ask', 'questions', 'if','have', 'any', 'math', 'not', 'good', 'submit', 'pay'] 

test_emails=[
             ["hello students, please learn for the exam", 0],
             ["hello students, please buy drugs", 1],    
             ["hello, today the sun is shining in nagoya", 0],
             ["lets sell drugs in nagoya", 1],
             ["today learn drugs", 1],
             ["how are you today?", 0],
             ["hello students, please do your homework", 0],
             ["hello, do you want free homework solutions?", 1],
             ["hey students, please always ask questions if you have any", 0],
             ["math is not good", 1],
             ["math is good", 0],
             ["please submit your homework", 0],
             ["please buy questions", 1],
             ["please pay for the exam", 1]          
             ]

The feature space for our spam filter will be $\mathcal{X}=\{0,1\}^d$, where $d$ denotes the number of words in the dictionary. For a feature (email) $x \in \mathcal{X}$ the entry $x_i$ for $i=1,\dots,d$ is $1$ if the $i$-th word of the dictionary is contained in the email and $0$ otherwise.  

# **Exercise 1**
Implement a function which creates a feature vector out of an email and a function which creates a training set out of test emails. 

You would need to figure out whether a sentence contains a word, and there are functions in Python that could determine whether a string contains another string. You can consult documentation (and Google) to find out.

In [184]:
def email_to_feature(dict, email):
  feature = np.zeros(len(dict))
  for i in range(len(dict)):
    if dict[i] in email[0]:
      feature[i]=1
    else:
      feature[i]=0
  return feature

def emails_to_test_samples(dict, test_emails):
  test_samples = []
  for j in range(len(test_emails)):
    samples = email_to_feature(dictionary,test_emails[j])
    samples = list(map(int, samples))
    test_samples.append(samples)
    df_samples = pd.DataFrame(test_samples, columns=dictionary)
  return df_samples
  # raise NotImplementedError

print(email_to_feature(dictionary,test_emails[0]))
print('=====================================================')
df_samples = emails_to_test_samples(dict, test_emails)
print(df_samples)

for k in range(len(test_emails)):
  del test_emails[k][0]
df_spams_first = pd.DataFrame(test_emails)
df_spams = df_spams_first.rename(columns={0: 'spam'})
print(df_spams)

[1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
    hello  students  please  learn  for  the  ...  any  math  not  good  submit  pay
0       1         1       1      1    1    1  ...    0     0    0     0       0    0
1       1         1       1      0    0    0  ...    0     0    0     0       0    0
2       1         0       0      0    0    1  ...    0     0    0     0       0    0
3       0         0       0      0    0    0  ...    0     0    0     0       0    0
4       0         0       0      1    0    0  ...    0     0    0     0       0    0
5       0         0       0      0    0    0  ...    0     0    0     0       0    0
6       1         1       1      0    0    0  ...    0     0    0     0       0    0
7       1         0       0      0    0    0  ...    0     0    0     0       0    0
8       0         1       1      0    0    0  ...    1     0    0     0       0    0
9       0         0       0   

In [185]:
# In this cell, I want to make a DataFrame of spam-included words

df_feature = pd.concat([df_samples, df_spams], axis=1)
df_spams_feature = df_feature[df_feature['spam'] == 1]
df_spams_words = df_spams_feature.drop('spam', axis=1)
df_spams_words
#print(df_feature)

,hello,students,please,learn,for,the,exam,buy,drugs,today,sun,is,shining,in,nagoya,lets,sell,how,are,you,today?,do,your,homework,want,free,solutions?,hey,always,ask,questions,if,have,any,math,not,good,submit,pay
1,1,1,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0
12,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
13,0,0,1,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [186]:
# In thsi cell, I want to cluculate how likely these words appere in spam-mails
spam = df_feature['spam'][df_feature["spam"] == 1].count()
not_spam = df_feature['spam'][df_feature["spam"] == 0].count()
print(spam)
print(not_spam)

print(df_spams_words.sum()/spam)

7
7
hello         0.285714
students      0.142857
please        0.428571
learn         0.142857
for           0.142857
the           0.142857
exam          0.142857
buy           0.285714
drugs         0.428571
today         0.142857
sun           0.000000
is            0.142857
shining       0.000000
in            0.142857
nagoya        0.142857
lets          0.142857
sell          0.142857
how           0.000000
are           0.000000
you           0.142857
today?        0.000000
do            0.142857
your          0.000000
homework      0.142857
want          0.142857
free          0.142857
solutions?    0.142857
hey           0.000000
always        0.000000
ask           0.000000
questions     0.142857
if            0.000000
have          0.000000
any           0.000000
math          0.142857
not           0.142857
good          0.142857
submit        0.000000
pay           0.142857
dtype: float64


In [187]:
# In this cell, I want to make a DataFrame of spam-NOT-included words

df_notspams_feature = df_feature[df_feature['spam'] == 0]
df_notspams_words = df_notspams_feature.drop('spam', axis=1)
df_notspams_words

,hello,students,please,learn,for,the,exam,buy,drugs,today,sun,is,shining,in,nagoya,lets,sell,how,are,you,today?,do,your,homework,want,free,solutions?,hey,always,ask,questions,if,have,any,math,not,good,submit,pay
0,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,1,0,0,0,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,1,1,1,1,1,0,0,0,0,0
10,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0
11,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [188]:
# In thsi cell, I want to cluculate how likely these words appere in NOT-spam-mails
print(df_notspams_words.sum()/not_spam)

hello         0.428571
students      0.428571
please        0.571429
learn         0.142857
for           0.142857
the           0.285714
exam          0.142857
buy           0.000000
drugs         0.000000
today         0.285714
sun           0.142857
is            0.285714
shining       0.142857
in            0.142857
nagoya        0.142857
lets          0.000000
sell          0.000000
how           0.142857
are           0.142857
you           0.571429
today?        0.142857
do            0.142857
your          0.285714
homework      0.285714
want          0.000000
free          0.000000
solutions?    0.000000
hey           0.142857
always        0.142857
ask           0.142857
questions     0.142857
if            0.142857
have          0.142857
any           0.142857
math          0.142857
not           0.000000
good          0.142857
submit        0.142857
pay           0.000000
dtype: float64


spamにおいて1と0の総数を求める。

それぞれの単語においてspamでその単語が出てきた場合、(1でのその単語数)/(1の総数)をそれぞれの単語について求め、新しいベクトルを作ってみる。

①：1 or 0の数/総数

②：各単語の条件付確率

③：①×②の積

 **Recall from Lecture 6:**

Given a training set  $\mathcal{T} = \left( (x^{(1)}, y^{(1)}) , \dots, (x^{(n)}, y^{(n)})   \right)$ we calculate for $i=1,\dots,d$ the following numbers
\begin{align*}
\phi_{i\mid y=1} &= \frac{1+\sum_{j=1}^n I(x^{(j)}_i = 1  \,\wedge \, y^{(j)}=1 ) }{2+\sum_{j=1}^n I(y^{(j)}=1)}\,,\\
	\phi_{i\mid y=0} &= \frac{1+\sum_{j=1}^n I(x^{(j)}_i = 1  \,\wedge \, y^{(j)}=0 )}{2+\sum_{j=1}^n I(y^{(j)}=0)}\,,\\
		\phi_{y=1} &= \frac{1+\sum_{j=1}^n I(y^{(j)} = 1)}{2+n} \,.
\end{align*}
Here $I$ denoted the indicator function. We used the laplace smoothing (thats why we have the $1+$ and $2+$) in order to make sure that we will not assume probability $0$ for unknown words.

Now assume we get a new feature (i.e. someone sends us an email) $x \in \{0,1\}^d$. Then we can calculate for each word $i=1,\dots,d$ the probabilities
\begin{align*}
P(x_i = 1 \mid y=1) &= \phi_{i\mid y=1}\,,\qquad &&P(x_i = 1 \mid y=0) = \phi_{i\mid y=0} \,,\\
P(x_i = 0 \mid y=1) &= 1- \phi_{i\mid y=1}\,,\qquad &&P(x_i = 0 \mid y=0) = 1-\phi_{i\mid y=0} \,. \\
\end{align*}

By the Naive Bayes assumption we have for $x \in \{0,1\}^d$
\begin{align*}
P(x \mid y)  = \prod_{i=1}^d P(x_j \mid y)\,.
\end{align*}
The probability of the new email being spam is then
\begin{align*}
P(y=1 \mid x) &= \frac{ P(x \mid y=1) P(y=1)}{P(x)}\\
&= \frac{\prod_{i=1}^d P(x_i \mid y = 1) \cdot \phi_{y=1}}{\prod_{i=1}^d P(x_i \mid y = 1) \cdot \phi_{y=1} + \prod_{i=1}^d P(x_i \mid y = 0) (1-\phi_{y=1})}\,.
\end{align*}


# **Exercise 2:** 
Use the above explanation of the Naive Bayes Spam filter and implement a function which gives the probability of an email being spam given the trainings email above. 


In [194]:
# You can implement the above algorithm in any way you want.
# One possible way:
# - Calculate the phis here
# - Write functions for the probability P(x|y) depending on these phis
# - Use this function in the function spam_percentage 
# 
# def spam_percentage(email):
# 
#   return NotImplementedError

spam_mails_first = pd.DataFrame(df_spams_words.sum()/spam)
spam_mails = spam_mails_first.rename(columns={0: 'spam'})
not_spam_mails_first = pd.DataFrame(df_notspams_words.sum()/not_spam)
not_spam_mails = not_spam_mails_first.rename(columns={0: 'not_spam'})

spam_or_not_first = pd.concat([spam_mails, not_spam_mails], axis=1)
spam_or_not = spam_or_not_first.replace({0: 0.01})
spam_or_not

,spam,not_spam
hello,0.285714,0.428571
students,0.142857,0.428571
please,0.428571,0.571429
learn,0.142857,0.142857
for,0.142857,0.142857
the,0.142857,0.285714
exam,0.142857,0.142857
buy,0.285714,0.010000
drugs,0.428571,0.010000
today,0.142857,0.285714


Test your spam filter with the following email.


In [193]:
email="the sun is shining. buy drugs now"

# Thsi is too difficult for me, please let me caluculate without using algorithm.

spam = not spam = 7/14 = 0.5

the : {spam:0.142857, not spam:0.285714}

sun: {spam:0.01, not spam:0.142857}

is: {spam:0.142857, not spam:0.285714}

shining: {spam:0.01, not spam:0.142857}

buy: {spam:0.285714, not spam:0.01}

drugs: {spam:0.428571, not spam:0.01}

now: {1}

Spam-Possibility = 1.2494...$\times e^-7$

Not-Spam-Possibility = 8.3298...$\times e^-8$

Spam-Possibility is more than Not-Spam-Possibility

then answer is **SPAM**


# **Exercise 3**
Extend your spamfilter by creating a dynamical dictionary. Instead of starting with a fixed dictionary, you should now create a dictionary out of a list of emails. 

Write a function `create_dictionary(emails)` which returns a dictionary created from a list of emails (Give as an array of arrays `[text, spam\nospam]`). Make sure that you do not include words more than once into the dictionary.
To implement this function you should look up the function `split()` for a string in Python. To take care of the symbols "." and "," you can use the `replace()` function of a string.

In [ ]:
def create_dictionary(emails):

  return NotImplementedError